In [ ]:
%pip install mlxtend

In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
df = spark.sql("SELECT * FROM Prod_SB_Lakehouse.Market_Basket_Analysis_from_SQL") 
display(df.head(3))

In [ ]:
print("table size",df.count())

In [ ]:
df = df.toPandas()

In [ ]:
df.dtypes

In [ ]:
df = df[["Account","Item_GBL","Quantity", "YearPeriod"]]
df["Account_Month"] = df["Account"].astype(str) + df["YearPeriod"].astype(str)
df = df.convert_dtypes()
df["Account_Month"] = df["Account_Month"].astype("category")
df["Item_GBL"] = df["Item_GBL"].astype("category")
df.dtypes

In [ ]:
df_items_frequency = df.groupby("Item_GBL")["Account_Month"].count()
list_of_items = df_items_frequency[df_items_frequency >= 200].index.tolist()
df = df[df["Item_GBL"].isin(list_of_items)]
del df_items_frequency, list_of_items

print("New table size",df.shape)

In [ ]:
df["Quantity"] = 1
df = df.drop_duplicates()

In [ ]:
df = df[["Account_Month","Item_GBL","Quantity"]]

from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype
import numpy as np
import pandas as pd

# As we have categorical data but sparse matrixes don't love it I map categories to Int
item_c = CategoricalDtype(sorted(df["Item_GBL"].unique()), ordered=True)
invoice_c = CategoricalDtype(sorted(df["Account_Month"].unique()), ordered=True)

row = df["Account_Month"].astype(invoice_c).cat.codes
col = df["Item_GBL"].astype(item_c).cat.codes

#Create Sparse matrix
sparse_matrix = csr_matrix((df["Quantity"], (row, col)), \
                           shape=(invoice_c.categories.size, item_c.categories.size))
    
# Replace NAN in Sparse Matrix to Zeroes
sparse_matrix = np.nan_to_num(sparse_matrix, copy=False)

df = pd.DataFrame.sparse.from_spmatrix(sparse_matrix)       



In [ ]:
from mlxtend.frequent_patterns import fpgrowth

frequent_itemsets = fpgrowth(df, min_support=0.003, use_colnames=True)

from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

In [ ]:
display(frequent_itemsets.tail(10))


In [ ]:
display(rules.head(10))

In [ ]:
print("\n"*2, "Total rules found", rules.shape[0], "\n"*3)

tmp = rules.select_dtypes(include=[np.number])
rules.loc[:, tmp.columns] = np.round(tmp,5)

display(rules.sort_values(["support","confidence"], ascending=False).head(25))

In [ ]:
rules[['antecedents','consequents']] = rules[['antecedents','consequents']].applymap(lambda x: tuple(x))
rules['combined_itemset'] = rules['antecedents'] + rules['consequents']
rules['combined_itemset'] = [sorted(x) for x in rules['combined_itemset']]

def tuple_to_str(insert_tuple):
    string = ''
    for element in insert_tuple:
        string = string + str(element) + ' '
    return string
      
rules['combined_itemset'] = rules['combined_itemset'].apply(tuple_to_str)
rules['antecedents'] = rules['antecedents'].apply(tuple_to_str)
rules['consequents'] = rules['consequents'].apply(tuple_to_str)

In [ ]:
rules.to_parquet('//lakehouse/default/Files/Association_Rules_Mined.parquet')

df_mapped_items = pd.DataFrame({'items':item_c.categories})
df_mapped_items['item_code'] = df_mapped_items["items"].astype(item_c).cat.codes

df_mapped_items.to_parquet('//lakehouse/default/Files/Items_Mapped_to_Num.parquet')


